# $k$-Nearest Neighbors

In [1]:
# imports

import pandas as pd
import numpy as np

-(np.log2(2/3)*(2/3) + np.log2(1/3)*(1/3))

0.9182958340544896

## Save Data To Files For CV

In [4]:
df = pd.read_csv('processed.cleveland.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [5]:
df.shape

(303, 14)

In [6]:
# num is the label column

features = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'] # from heart-disease.names number 7

In [7]:
df.columns = features

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [8]:
# cast label to binary

df['num'] = df['num'].apply(lambda x: 1 if x != 0 else 0)

In [18]:
# features 1, 2, 5, 6, 8, 10, 11, 12

df['thal'].value_counts()

3.0    166
7.0    117
6.0     18
?        2
Name: thal, dtype: int64

In [11]:
df['num'].value_counts()

0    164
1    139
Name: num, dtype: int64

In [26]:
# drop missing values 

df = df[(df.ca != '?') & (df.thal != '?')]
df.shape

(297, 15)

In [30]:
# cast to float

df['ca'] = df.ca.astype('float64')
df['thal'] = df.thal.astype('float64')

In [31]:
data_X = df.iloc[:, :-1].to_numpy()
data_X.shape

(297, 14)

In [32]:
data_Y = df.iloc[:, -1].to_numpy()
data_Y.shape

(297,)

In [33]:
# enumerate 0 ... 9 repeatedly until length of df

from itertools import cycle

lst_cycle = cycle(list(range(10)))
enum = [next(lst_cycle)for i in range(len(df))]

In [34]:
# permute samples

df = df.sample(frac=1, random_state=42)

In [35]:
# save to 10 different npy files

df['enum'] = enum
for i, group in df.groupby('enum'):
    np.savez(f'heart_fold{i}', x=group.iloc[:, :-2].to_numpy(), y=group['num'].to_numpy(), example_names=group.index.to_numpy())

## Naive KNN Algorithm

In [3]:
test_data = np.load('heart_fold0.npz', allow_pickle=True)
X = test_data['x']
y = test_data['y']
example_names = test_data['example_names']

In [4]:
# confirm shapes look reasonable

X.shape, y.shape, example_names.shape

((30, 13), (30,), (30,))

In [5]:
from collections import Counter
import numpy as np

class NaiveKNNClassifier:
    def __init__(self, k):
        self.k = k
        self.data = None

    def fit(self, X_train: np.array, y_train: np.array):
        self.data = X_train
        labels = y_train
        self.labels = np.where(labels == 0, labels, 1) # cast to boolean

    def predict_one(self, test_example: np.array):
        assert self.data is not None
        assert test_example.shape[0] == self.data.shape[1] # assert that arrays are broadcastable
        norms = np.linalg.norm(test_example - self.data, axis=1)
        sorted_norm_idx = np.argsort(norms)
        sorted_norm_idx = sorted_norm_idx[:self.k]
        common_counter = Counter(self.labels[sorted_norm_idx])
        return common_counter.most_common(1)[0][0]

    def predict(self, test_examples: np.array):
        assert self.data is not None
        assert test_examples.shape[1] == self.data.shape[1] # assert that arrays are broadcastable
        return np.apply_along_axis(self.predict_one, 1, test_examples)
        

# unit test
clf = NaiveKNNClassifier(3)
clf.fit(np.array([[3, 4], [1, 2], [5, 6], [7, 8]]), np.array([1, 1, 0, 0]))
clf.predict(np.array([[1, 1], [2, 2], [7, 7]]))

array([1, 1, 0])

## CV with KNN

1. Implement the kNN algo (choose whatever distance function you want)
2. Treat each of your 10 data folds from HW0 as a test set and
compute the accuracy of kNN on it (and report the chosen k)
3. Use the other 9 folds to choose a good k
4. Consider k in {1, 3, 7, 15, 25, 51, 101}; each test set might get a different k
5. Nine times, use 8 of these 9 folds as the train set and 
the 9th as the tune set.   Find which one k works best across all 9
6. Then use all 9 folds as the train set plus the best k 
to compute the accuracy on the fold that is the test set

### Back of the Envelope Calculation:

1. Assume 10 folds and 10n total examples
10 train-test folds and, for each of them, 9 train’-tune folds
2. How many times will distanceSq(exA, exB) be called?

(Assuming naive implementation of knn) per tune fold, $D$ will be called $8n \cdot n \cdot m$ times, where $m$ is the number of values of $k$ to try.
Then per test fold, $D$ will be called $n \cdot 9n$ times. Therefore $D$ is called a total of $720n^2m + 90n^2$ times.

10x
    per tune fold have to do 8n*n 
    hence in total, 72n^2
    +
    9n^2 
    = 
    81n^2
hence 810n^2

additional hw assignment: modify to cache then add a counter to see if number matches up with theoretical calculation

In [6]:
# utility functions for loading files

from typing import List

def load_one_file(filename: str):
    data = np.load(filename, allow_pickle=True)
    X = data['x']
    y = data['y']
    example_names = data['example_names']
    return X, y, example_names

def load_folds(folds: List[int]):
    all_X = []
    all_y = []
    all_example_names = []
    for fold in folds:
        filename = f'heart_fold{fold}.npz'
        X, y, example_names = load_one_file(filename)
        all_X.append(X)
        all_y.append(y)
        all_example_names.append(example_names)

    X_folds = np.concatenate(all_X, axis=0)
    y_folds = np.concatenate(all_y)
    example_names_folds = np.concatenate(all_example_names)
        
    return X_folds, y_folds, example_names_folds

X, y, example_names = load_folds([0, 1])
X.shape, y.shape, example_names.shape

((60, 13), (60,), (60,))

In [7]:
# utility functions for calculating and displaying metrics

def accuracy_score(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

In [12]:
from tqdm import trange

def get_tune_results(train_tune_folds: List[int], k_values=[1, 3, 7, 15, 25, 101]):
    results_dict = {k: [] for k in k_values}
    for tune_fold in trange(len(train_tune_folds)):
        X_tune, y_tune, example_names_tune = load_folds([tune_fold])
        train_folds = [fold for fold in train_tune_folds if fold != tune_fold]
        X_train, y_train, example_names_train = load_folds(train_folds)

        for k in k_values:
            clf = NaiveKNNClassifier(k)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_tune)
            score = accuracy_score(y_tune, y_pred)
            results_dict[k].append(score)
        
    return pd.DataFrame(results_dict)

def get_optimal_k(tune_results: pd.DataFrame):
    return tune_results.columns[tune_results.mean().argmax()], tune_results.mean()


def cv(num_folds=10):
    cv_results = []
    ks = []
    for test_fold in trange(num_folds):
        X_test, y_test, example_names_test = load_folds([test_fold])
        train_folds = [fold for fold in range(num_folds) if fold != test_fold]
        tune_results = get_tune_results(train_folds)
        print(f'tune accuracy matrix, fold {test_fold}: \n{tune_results}')
        k, mean_results = get_optimal_k(tune_results)
        ks.append(k)
        print(f'average accuracy for each k, fold {test_fold}: \n{mean_results}')
        print(f'optimal k for fold {test_fold}: {k}')
        clf = NaiveKNNClassifier(k)
        X_train, y_train, example_names_train = load_folds(train_folds)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        cv_results.append(acc)

    print(ks)
    return cv_results

cv_scores = cv()
print(f'average cv score: {np.mean(cv_scores)}')

 10%|█         | 1/10 [00:00<00:04,  2.23it/s]

tune accuracy matrix, fold 0: 
        1         3         7         15        25        101
0  0.633333  0.566667  0.666667  0.600000  0.600000  0.533333
1  0.433333  0.433333  0.733333  0.666667  0.733333  0.666667
2  0.566667  0.700000  0.633333  0.633333  0.600000  0.633333
3  0.433333  0.566667  0.600000  0.500000  0.666667  0.633333
4  0.533333  0.666667  0.666667  0.633333  0.633333  0.666667
5  0.700000  0.666667  0.700000  0.700000  0.666667  0.500000
6  0.600000  0.633333  0.633333  0.566667  0.666667  0.533333
7  0.586207  0.620690  0.655172  0.689655  0.758621  0.689655
8  0.793103  0.793103  0.793103  0.689655  0.793103  0.689655
average accuracy for each k, fold 0: 
1      0.586590
3      0.627458
7      0.675734
15     0.631034
25     0.679821
101    0.616220
dtype: float64
optimal k for fold 0: 25


 20%|██        | 2/10 [00:00<00:03,  2.43it/s]

tune accuracy matrix, fold 1: 
        1         3         7         15        25        101
0  0.600000  0.566667  0.700000  0.666667  0.600000  0.533333
1  0.466667  0.500000  0.666667  0.633333  0.700000  0.633333
2  0.600000  0.700000  0.633333  0.633333  0.600000  0.700000
3  0.400000  0.500000  0.500000  0.500000  0.633333  0.666667
4  0.600000  0.700000  0.633333  0.633333  0.633333  0.633333
5  0.600000  0.633333  0.633333  0.633333  0.700000  0.533333
6  0.566667  0.633333  0.566667  0.533333  0.633333  0.533333
7  0.620690  0.655172  0.724138  0.689655  0.724138  0.689655
8  0.793103  0.793103  0.862069  0.724138  0.862069  0.793103
average accuracy for each k, fold 1: 
1      0.583014
3      0.631290
7      0.657727
15     0.627458
25     0.676245
101    0.635121
dtype: float64
optimal k for fold 1: 25


 30%|███       | 3/10 [00:01<00:02,  2.38it/s]

...

tune accuracy matrix, fold 7: 
        1         3         7         15        25        101
0  0.566667  0.533333  0.666667  0.600000  0.500000  0.566667
1  0.500000  0.600000  0.666667  0.633333  0.666667  0.666667
2  0.600000  0.633333  0.700000  0.633333  0.666667  0.633333
3  0.366667  0.533333  0.533333  0.533333  0.633333  0.666667
4  0.500000  0.666667  0.633333  0.633333  0.633333  0.666667
5  0.700000  0.733333  0.700000  0.700000  0.733333  0.533333
6  0.666667  0.633333  0.633333  0.600000  0.666667  0.566667
7  0.620690  0.655172  0.758621  0.724138  0.724138  0.724138
8  0.758621  0.724138  0.827586  0.758621  0.827586  0.689655
average accuracy for each k, fold 7: 
1      0.586590
3      0.634738
7      0.679949
15     0.646232
25     0.672414
101    0.634866
dtype: float64
optimal k for fold 7: 7


 90%|█████████ | 9/10 [00:03<00:00,  2.54it/s]

tune accuracy matrix, fold 8: 
        1         3         7         15        25        101
0  0.600000  0.533333  0.666667  0.566667  0.500000  0.500000
1  0.466667  0.500000  0.666667  0.700000  0.666667  0.633333
2  0.566667  0.666667  0.666667  0.633333  0.566667  0.633333
3  0.433333  0.533333  0.533333  0.566667  0.700000  0.600000
4  0.533333  0.666667  0.566667  0.633333  0.666667  0.666667
5  0.633333  0.733333  0.766667  0.700000  0.733333  0.533333
6  0.533333  0.600000  0.633333  0.566667  0.566667  0.533333
7  0.586207  0.655172  0.689655  0.689655  0.689655  0.724138
8  0.793103  0.758621  0.827586  0.689655  0.827586  0.724138
average accuracy for each k, fold 8: 
1      0.571775
3      0.627458
7      0.668582
15     0.638442
25     0.657471
101    0.616475
dtype: float64
optimal k for fold 8: 7


100%|██████████| 10/10 [00:04<00:00,  2.48it/s]

tune accuracy matrix, fold 9: 
        1         3         7         15        25        101
0  0.633333  0.600000  0.600000  0.633333  0.633333  0.533333
1  0.500000  0.500000  0.700000  0.600000  0.733333  0.600000
2  0.566667  0.700000  0.633333  0.666667  0.600000  0.733333
3  0.433333  0.533333  0.600000  0.566667  0.633333  0.633333
4  0.566667  0.633333  0.600000  0.666667  0.600000  0.700000
5  0.600000  0.666667  0.700000  0.733333  0.733333  0.533333
6  0.600000  0.633333  0.600000  0.566667  0.600000  0.566667
7  0.620690  0.655172  0.689655  0.689655  0.758621  0.758621
8  0.793103  0.655172  0.758621  0.758621  0.862069  0.827586
average accuracy for each k, fold 9: 
1      0.590421
3      0.619668
7      0.653512
15     0.653512
25     0.683780
101    0.654023
dtype: float64
optimal k for fold 9: 25
[25, 25, 7, 7, 7, 7, 25, 7, 7, 25]
average cv score: 0.6877011494252873
